<a href="https://colab.research.google.com/github/566aass/colab/blob/colab_code/silo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
# 挂载 Google 云盘
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn==1.3.0  # 安装 scikit-learn，包含 train_test_split
!pip install tensorflow==2.13.0     # 安装 TensorFlow，keras 的后端
!pip install keras==2.13.1       # 安装 Keras
!pip install numpy==1.25.2        # 安装 NumPy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.3.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: t

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.3.0 which is incompatible.
nibabel 5.3.2 requires typing-extensions>=4.6; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.25.2 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.13.0 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LeakyReLU,Dropout,ReLU
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# 数据预处理
def preprocess_data(data):
    dataset = data

    # 将数据划分为训练集和测试集
    # 注意：这里假设目标变量是每个样本对应的最大降雨侵蚀月份（0-11）
    scaler = MinMaxScaler()
    X = scaler.fit_transform(dataset)
    y = np.argmax(dataset, axis=1)  # 目标变量，表示最大降雨侵蚀月份（0-11）

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return x_train, x_test, y_train, y_test


def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, input_shape=input_shape))
    model.add(ReLU())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(ReLU())
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(ReLU())
    model.add(Dense(1, activation='linear'))
    optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=1e-6, nesterov=True)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'accuracy'])
    # model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

# 读取数据文件，并且筛选数据，然后转换数据维度
df = pd.read_csv('/content/drive/MyDrive/Neu/silo.csv')

# 筛选数据
data = df.iloc[:, 3:].fillna(0).values  # 使用 0 填充缺失值

# 拆分数据
x_train, x_test, y_train, y_test = preprocess_data(data)
y_train_onehot = to_categorical(y_train, num_classes=12)
y_test_onehot = to_categorical(y_test, num_classes=12)

# 创建模型
input_shape = (x_train.shape[1], 1)
model = create_model(input_shape)

# 进行模型训练
model.fit(x_train, y_train_onehot, epochs=10, batch_size=64, validation_split=0.2)
loss, mae, accuracy = model.evaluate(x_test, y_test_onehot, verbose=0)

print(f"Test Loss: {loss:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# 获取最后一层神经网络的权重，其形状为(12, 12)
# weights = model.layers[-1].get_weights()[0]
# # 将权重转换为一维数组
# weights_array = weights.flatten()
# print(weights_array)
# # 获取权重绝对值最大的索引
# label_index = np.argmax(np.abs(weights_array))
# # 根据索引值获取对应的月份
# max_weight_index = label_index % 12 + 1
# # 构建标签与月份的映射关系
# month_mapping = {1: '一月', 2: '二月', 3: '三月', 4: '四月', 5: '五月', 6: '六月',
#                  7: '七月', 8: '八月', 9: '九月', 10: '十月', 11: '十一月', 12: '十二月'}
# # 根据索引值获取对应的月份
# target_month = month_mapping[max_weight_index]

# print('降雨侵蚀最大的月份：', target_month)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9167 - loss: 0.0788 - mae: 0.1422 - val_accuracy: 0.9167 - val_loss: 0.0765 - val_mae: 0.1542
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9167 - loss: 0.0768 - mae: 0.1518 - val_accuracy: 0.9167 - val_loss: 0.0765 - val_mae: 0.1542
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9167 - loss: 0.0767 - mae: 0.1525 - val_accuracy: 0.9167 - val_loss: 0.0765 - val_mae: 0.1545
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9167 - loss: 0.0766 - mae: 0.1525 - val_accuracy: 0.9167 - val_loss: 0.0765 - val_mae: 0.1558
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9167 - loss: 0.0766 - mae: 0.1528 - val_accuracy: 0.9167 - val_loss: 0.0764 - val_mae: 0.1548
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9167 - loss: 0.0766 - mae: 0.1525 - val_accuracy: 0.9167 - val_loss: 0.0764 - val_mae: 0.1554
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step -

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# 加载数据
df = pd.read_csv('/content/drive/MyDrive/Neu/silo.csv')

# 筛选数据并填充缺失值
data = df.iloc[:, 3:].fillna(0).values

def preprocess_data(data):
    # 特征缩放
    scaler = MinMaxScaler()
    X = scaler.fit_transform(data)

    # 目标变量：降雨侵蚀最大的月份索引 (0-11)
    y = np.argmax(data, axis=1)

    # 划分训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 将训练集和测试集转换为 RNN 模型所需的形状 (samples, timesteps, features)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test


def create_rnn_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, input_shape=input_shape))
    model.add(Dense(1, activation='linear'))

    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model
# 设置随机种子
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# 数据预处理
x_train, x_test, y_train, y_test = preprocess_data(data)

# 创建模型
input_shape = (x_train.shape[1], 1)  # (12, 1)
print(input_shape)
model = create_rnn_model(input_shape)

# 训练模型
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# 预测
predictions = model.predict(x_test)

# 将预测结果转换为月份 (0-11 -> 1-12)
predicted_months = [int(pred[0]) + 1 for pred in predictions]
predicted_labels = [month - 1 for month in predicted_months]

# 计算准确率
accuracy = accuracy_score(y_test, predicted_labels)

# 计算精确率 (macro average)
precision = precision_score(y_test, predicted_labels, average='macro')

# 计算召回率 (macro average)
recall = recall_score(y_test, predicted_labels, average='macro')

# 计算 F1 分数 (macro average)
f1 = f1_score(y_test, predicted_labels, average='macro')

# 打印指标
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 计算混淆矩阵
confusion = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(confusion)


(12, 1)
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 37.0706 - mae: 4.6841 - val_loss: 26.3703 - val_mae: 4.9548
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 25.6177 - mae: 4.9032 - val_loss: 26.3054 - val_mae: 4.9448
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 25.5546 - mae: 4.8979 - val_loss: 26.1766 - val_mae: 4.9462
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 25.3324 - mae: 4.8817 - val_loss: 23.6512 - val_mae: 4.6120
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 23.5326 - mae: 4.5714 - val_loss: 23.7026 - val_mae: 4.3862
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 22.7344 - mae: 4.3445 - val_loss: 22.6076 - val_mae: 4.3666
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 22.3295 - mae: 4.3616 - val_loss: 22.5681 - val_mae: 4.1900
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 22.1210 - mae: 4.3212 - val_loss: 21.9408 - val_mae: 4.1949
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score # 导入cohen_kappa_score



# 加载数据
df = pd.read_csv('/content/drive/MyDrive/Neu/silo.csv')

# 筛选数据并填充缺失值
data = df.iloc[:, 3:].fillna(0).values

# 目标变量：降雨侵蚀最大的月份索引 (0-11)
target = np.argmax(data, axis=1)

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
print(np.unique(target))
# 创建随机森林分类器
model = RandomForestClassifier(n_estimators=150, random_state=42)

# 训练模型
model.fit(x_train, y_train)

# 预测
predictions = model.predict(x_test)

# 计算准确率
overall_accuracy = accuracy_score(y_test, predictions)

# 计算 Kappa 系数
kappa = cohen_kappa_score(y_test, predictions)

# 计算混淆矩阵
confusion_mat = confusion_matrix(y_test, predictions)

# 打印指标
print(f"总体精度: {overall_accuracy}")
print(f"Kappa 系数: {kappa}")
print(f"混淆矩阵:\n{confusion_mat}")


[ 0  1  2  3  9 10 11]
总体精度: 0.943
Kappa 系数: 0.9163203257928649
混淆矩阵:
[[252  11   0   0   1  11]
 [  3 320   0   0   0   6]
 [  0   2  22   0   0   0]
 [  0   0   0   0   0   2]
 [  1   1   0   1   3   0]
 [  9   8   1   0   0 346]]
